In [ ]:
#Sentiment Analysis con BERT

from transformers import AutoTokenizer

model = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model, model_max_length=512)

def preprocessing_function(examples):
   return tokenizer(examples['text'], truncation=True)

from datasets import load_dataset
imdb = load_dataset("imdb")


small_tr = tokenized_imdb['train'].select(range(7500))
small_ts = tokenized_imdb['test'].select(range(5000))


from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")


DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path="prajjwal1/bert-tiny",
    vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right',
    truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]',
    'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'},
    clean_up_tokenization_spaces=True), padding=True, max_length=None,
    pad_to_multiple_of=None, return_tensors="tf")


from transformers import TFAutoModelForSequenceClassification


id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

tf_model = TFAutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2, id2label=id2label,
    label2id=label2id, from_pt=True
)


tf_train_set = tf_model.prepare_tf_dataset(
    small_tr,
    shuffle = True,
    batch_size=32,
    collate_fn = data_collator
)

tf_validation_set = tf_model.prepare_tf_dataset(
    small_ts,
    shuffle = False,
    batch_size=32,
    collate_fn = data_collator
)



tf_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5))

import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_validation_set)

tf_model.fit(x = tf_train_set, validation_data = tf_validation_set, epochs = 3, callbacks = [metric_callback])

tf_model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=metric_callback)



from transformers import TextClassificationPipeline
text = "I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood."
classifier = TextClassificationPipeline(tokenizer=tokenizer, model=tf_model, return_all_scores=True)
classifier(text)

